In [3]:
#conda install langchain -c conda-forge
#conda install langchain langchain-community langchain-chroma -c conda-forge

from langchain_community.llms import Ollama
llm = Ollama(model="llama3")
response = llm.invoke("The first man on the moon was ...")
print(response)

A classic question!

The first humans to walk on the Moon were Neil Armstrong and Edwin "Buzz" Aldrin. They landed on July 20, 1969, as part of the Apollo 11 mission. Armstrong stepped out of the lunar module Eagle and onto the Moon's surface at 2:56 UTC on July 21, famously declaring, "That's one small step for man, one giant leap for mankind." Aldrin joined him on the surface a few minutes later.


Read transaction data

In [4]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/thu-vu92/local-llms-analyse-finance/refs/heads/main/transactions_2022_2023.csv")
df

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08
7,2023-12-22,Consulting,Income,541.57
8,2023-12-22,Aidsfonds,Expense,10.70
9,2023-12-20,Consulting,Income,2641.93


In [5]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [6]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

# Categorise bank transactions with Llama2

In [7]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [8]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [9]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Here are the added categories:', '', '* ISS Catering Services De Meern: Food', '* Taxi Utrecht: Transportation', '* Etos AMSTERDAM NLD: Health', '* Bistro Bar Amsterdam: Entertainment', '', 'So the categorized expenses would be:', '', '* Spotify AB by Adyen - Entertainment', '* Beta Boulders Ams Amsterdam Nld - Sport', '* ISS Catering Services De Meern - Food', '* Taxi Utrecht - Transportation', '* Etos AMSTERDAM NLD - Health', '* Bistro Bar Amsterdam - Entertainment']


,Transaction vs category,Transaction,Category
0,Here are the added categories:,Here are the added categories:,None
1,,,None
2,* ISS Catering Services De Meern: Food,* ISS Catering Services De Meern: Food,None
3,* Taxi Utrecht: Transportation,* Taxi Utrecht: Transportation,None
4,* Etos AMSTERDAM NLD: Health,* Etos AMSTERDAM NLD: Health,None
5,* Bistro Bar Amsterdam: Entertainment,* Bistro Bar Amsterdam: Entertainment,None
6,,,None
7,So the categorized expenses would be:,So the categorized expenses would be:,None
8,,,None
9,* Spotify AB by Adyen - Entertainment,* Spotify AB by Adyen,Entertainment


In [10]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

categories_df_all

['Here are the categorized expenses:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. Belastingdienst - Taxes', '4. Tesco Breda - Groceries', '5. Monthly Apartment Rent - Housing', '6. Vishandel Sier Amsterdam - Food', '7. Selling Paintings - Miscellaneous Income', '8. Tk Maxx Amsterdam Da - Clothing', '9. Consulting - Work', '10. Aidsfonds - Charity', '11. Tls Bv Inz Ov-Chipkaart - Transportation', '12. Etos Amsterdam - Health', '13. Salary - Employment', '14. Bouldermuur Bv Amsterdam - Entertainment', '15. Freelancing - Work', '16. Tikkie - Bills', '17. Blogging - Miscellaneous Income', '18. Taxi Utrecht - Transportation', '19. Apple Services - Technology', '20. Amazon Lux - Shopping', '21. Classpass* Monthly - Fitness', '22. Audible Uk AdblCo/Pymt Gbr - Entertainment']


,Transaction vs category,Transaction,Category
0,Here are the categorized expenses:,Here are the categorized expenses:,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
4,3. Belastingdienst - Taxes,3. Belastingdienst,Taxes
5,4. Tesco Breda - Groceries,4. Tesco Breda,Groceries
6,5. Monthly Apartment Rent - Housing,5. Monthly Apartment Rent,Housing
7,6. Vishandel Sier Amsterdam - Food,6. Vishandel Sier Amsterdam,Food
8,7. Selling Paintings - Miscellaneous Income,7. Selling Paintings,Miscellaneous Income
9,8. Tk Maxx Amsterdam Da - Clothing,8. Tk Maxx Amsterdam Da,Clothing


In [11]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [12]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Entertainment', 'Sport', 'Taxes', 'Groceries', 'Housing',
       'Food', 'Miscellaneous Income', 'Clothing', 'Work', 'Charity',
       'Transportation', 'Health', 'Employment', 'Bills', 'Technology',
       'Shopping', 'Fitness'], dtype=object)

In [13]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Fitness", then categorise as "Sport and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Fitness"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [ ]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

C:\Users\idani\AppData\Local\Temp\ipykernel_30252\3671457784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport and Fitness
4,3. Belastingdienst - Taxes,3. Belastingdienst,Taxes
5,4. Tesco Breda - Groceries,4. Tesco Breda,Groceries
6,5. Monthly Apartment Rent - Housing,5. Monthly Apartment Rent,Housing
7,6. Vishandel Sier Amsterdam - Food,6. Vishandel Sier Amsterdam,Food and Drinks
8,7. Selling Paintings - Miscellaneous Income,7. Selling Paintings,Miscellaneous Income
9,8. Tk Maxx Amsterdam Da - Clothing,8. Tk Maxx Amsterdam Da,Clothing
10,9. Consulting - Work,9. Consulting,Work
11,10. Aidsfonds - Charity,10. Aidsfonds,Charity


In [26]:
categories_df_all.reset_index(drop=True)
categories_df_all

,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport and Fitness
4,3. Belastingdienst - Taxes,3. Belastingdienst,Taxes
5,4. Tesco Breda - Groceries,4. Tesco Breda,Groceries
6,5. Monthly Apartment Rent - Housing,5. Monthly Apartment Rent,Housing
7,6. Vishandel Sier Amsterdam - Food,6. Vishandel Sier Amsterdam,Food and Drinks
8,7. Selling Paintings - Miscellaneous Income,7. Selling Paintings,Miscellaneous Income
9,8. Tk Maxx Amsterdam Da - Clothing,8. Tk Maxx Amsterdam Da,Clothing
10,9. Consulting - Work,9. Consulting,Work
11,10. Aidsfonds - Charity,10. Aidsfonds,Charity


In [27]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("https://raw.githubusercontent.com/thu-vu92/local-llms-analyse-finance/refs/heads/main/transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN
